### D)
Extend this and try to see how much of the bedroom you can reconstruct from the RGB and depth images.
You can extend a point cloud by `new = source + target`. Remember to resample the point cloud after a view additions so it does not get too large: `down_source = source.voxel_down_sample(voxel_size=0.05)`

In [46]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
import copy

from tqdm import tqdm

import glob
import os

In [47]:
rgb_paths = glob.glob('car_challange/rgb/*.jpg')
depth_paths = glob.glob('car_challange/depth/*.png')

In [48]:
rgb_paths.sort(key=lambda path: int(os.path.basename(path).split('.')[0]))
depth_paths.sort(key=lambda path: int(os.path.basename(path).split('.')[0]))

In [49]:
rgb_paths = rgb_paths[:100]
depth_paths = depth_paths[:100]

In [50]:
print(rgb_paths)

['car_challange/rgb/0000001.jpg', 'car_challange/rgb/0000002.jpg', 'car_challange/rgb/0000003.jpg', 'car_challange/rgb/0000004.jpg', 'car_challange/rgb/0000005.jpg', 'car_challange/rgb/0000006.jpg', 'car_challange/rgb/0000007.jpg', 'car_challange/rgb/0000008.jpg', 'car_challange/rgb/0000009.jpg', 'car_challange/rgb/0000010.jpg', 'car_challange/rgb/0000011.jpg', 'car_challange/rgb/0000012.jpg', 'car_challange/rgb/0000013.jpg', 'car_challange/rgb/0000014.jpg', 'car_challange/rgb/0000015.jpg', 'car_challange/rgb/0000016.jpg', 'car_challange/rgb/0000017.jpg', 'car_challange/rgb/0000018.jpg', 'car_challange/rgb/0000019.jpg', 'car_challange/rgb/0000020.jpg', 'car_challange/rgb/0000021.jpg', 'car_challange/rgb/0000022.jpg', 'car_challange/rgb/0000023.jpg', 'car_challange/rgb/0000024.jpg', 'car_challange/rgb/0000025.jpg', 'car_challange/rgb/0000026.jpg', 'car_challange/rgb/0000027.jpg', 'car_challange/rgb/0000028.jpg', 'car_challange/rgb/0000029.jpg', 'car_challange/rgb/0000030.jpg', 'car_chal

In [51]:
print(len(rgb_paths))
print(len(depth_paths))

100
100


In [52]:
target = None
trans_init = None

In [53]:
estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint()
threshold = 0.5
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

voxel_size = 0.01
voxel_size_2 = 0.05 
voxel_size_final = 0.005

In [54]:
point_cloud = o3d.geometry.PointCloud()

for i, (rgb_path, depth_path) in tqdm(enumerate(zip(rgb_paths, depth_paths)), total=len(rgb_paths)):

    rgb = o3d.io.read_image(rgb_path)
    depth = o3d.io.read_image(depth_path)
    
    # print(rgb_path)
    # print(depth_path)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        rgb, depth, convert_rgb_to_intensity=True)

    source = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, camera)

    source = source.voxel_down_sample(voxel_size)

    if target is not None:
        target.estimate_normals()

        icp_result = o3d.pipelines.registration.registration_icp(
            source, target, threshold, trans_init,
            estimation_method)

        # print(icp_result)
        trans_init = icp_result.transformation

        source.transform(trans_init)

        # source.paint_uniform_color(
        #     [1/len(rgb_paths) * (i+1), 1/len(rgb_paths) * (i+1), 1/len(rgb_paths) * (i+1)])

        point_cloud += source
        
    if i % 50 == 0:
        point_cloud = point_cloud.voxel_down_sample(voxel_size_2)

    target = source

    # point_cloud.transform([[1, 0, 0, 0],
    #                        [0, -1, 0, 0],
    #                        [0, 0, -1, 0], [0, 0, 0, 1]])

    # o3d.visualization.draw_geometries([point_cloud])

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]


In [55]:
point_cloud.points

std::vector<Eigen::Vector3d> with 2337741 elements.
Use numpy.asarray() to access data.

In [56]:
point_cloud.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

o3d.visualization.draw_geometries([point_cloud])